<a href="https://colab.research.google.com/github/aknsntn/visualisation/blob/main/Data_Tables_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Задание 1

In [ ]:
sales = pd.read_csv("drive/MyDrive/data/sales_train.csv", sep=',')
items = pd.read_csv("drive/MyDrive/data/items.csv", sep=',')

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).


In [ ]:
top = sales[['item_id', 'item_cnt_day']].groupby('item_id').sum().sort_values(by='item_cnt_day', ascending=False)
top['item_name'] = items['item_name']
ntop = top['item_name'].head(10)
ntop

item_id
20949    Corporate package T-shirt 1C Interest white (3...
2808               Diablo III [PC, Jewel, Russian version]
3732           Grand Theft Auto V [PS3, Russian subtitles]
17717                    Acceptance of funds for 1C-Online
5822     Playstation Store replenishment of wallet: Pay...
3734      Grand Theft Auto V [Xbox 360, Russian subtitles]
6675     Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...
3731            Grand Theft Auto V [PC, Russian subtitles]
1855                   Battlefield 4 [PC, Russian version]
16787                    Some of us [PS3, Russian version]
Name: item_name, dtype: object

2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
full_top=top.merge(sales, on=['item_id'], how='inner')
items_id = list(set(full_top[full_top.date_block_num < 12]['item_id']) & 
                set(full_top[(full_top.date_block_num > 11) & 
                             (full_top.date_block_num < 24)]['item_id']) & 
                set(full_top[full_top.date_block_num > 23]['item_id']))
items_id = list(set(items_id) & set(ntop.index))
top[top.index.isin(items_id)]['item_name']

item_id
20949    Corporate package T-shirt 1C Interest white (3...
2808               Diablo III [PC, Jewel, Russian version]
3732           Grand Theft Auto V [PS3, Russian subtitles]
17717                    Acceptance of funds for 1C-Online
5822     Playstation Store replenishment of wallet: Pay...
3734      Grand Theft Auto V [Xbox 360, Russian subtitles]
6675     Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...
1855                   Battlefield 4 [PC, Russian version]
16787                    Some of us [PS3, Russian version]
Name: item_name, dtype: object

# Задание 2

> При выполнении на pandas не используйте merge().

In [ ]:
item_categories = pd.read_csv('drive/MyDrive/data/item_categories.csv', sep=',')
top['category_id'] = items['category_id']
top_ctg = top.groupby('category_id').sum()
top_ctg['item_category_name'] = item_categories['item_category_name']
top_ctg = top_ctg.sort_values(by='item_cnt_day')
top_ctg

,item_cnt_day,item_category_name
category_id,,
51,1.0,Books - Cognitive literature
10,1.0,Game consoles - PS2
1,2.0,Accessories - PS2
53,3.0,Books - Fiction
52,3.0,Books - Guides
...,...,...
37,203284.0,Cinema - Blu-Ray
19,254887.0,Games - PS3
55,348591.0,Music - CD of local production



1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.

In [ ]:
min = top_ctg.head(2)
min.drop(columns='item_cnt_day')

,item_category_name
category_id,
51,Books - Cognitive literature
10,Game consoles - PS2


2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

In [ ]:
max = top_ctg.tail(1)
max.drop(columns='item_cnt_day')

,item_category_name
category_id,
40,Cinema - DVD


# Задание 3

1. Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами.

In [ ]:
shops = pd.read_csv('drive/MyDrive/data/shops.csv', sep=',')
top_shops_id = set(sales[['shop_id', 'item_cnt_day']].groupby('shop_id').sum().sort_values(by='item_cnt_day', ascending=False).head(5).index)
top_shops = items.merge(sales, on='item_id')
top_shops = top_shops[top_shops.shop_id.isin(top_shops_id)]

In [ ]:
top_shops_max = top_shops[top_shops.category_id.isin(max.index)]
top_shops_max = top_shops_max.groupby('item_id').mean().drop(columns =['category_id', 'date_block_num', 'shop_id', 'item_price'])
top_shops_max

,item_cnt_day
item_id,
0,1.000000
2,1.000000
3,1.000000
4,1.000000
5,1.000000
...,...
22157,1.000000
22159,1.142857
22160,1.000000


In [ ]:
top_shops_min = top_shops[top_shops.category_id.isin(min.index)]
top_shops_min # seems like they don't sell cognitive literature (category_id = 51) in cool shops.. 

,item_name,item_id,category_id,date,date_block_num,shop_id,item_price,item_cnt_day


2. Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.


In [ ]:
min_sales = top_shops.groupby('item_id').min().drop(columns =['category_id', 'date', 'shop_id', 'item_price', 'item_name', 'item_cnt_day'])
min_sales[min_sales.index.isin(top_shops_max.index)]

,date_block_num
item_id,
0,20
2,19
3,18
4,20
5,23
...,...
22157,4
22159,28
22160,0


In [ ]:
max_sales = top_shops.groupby('item_id').max().drop(columns =['category_id', 'date', 'shop_id', 'item_price', 'item_name', 'item_cnt_day'])
max_sales[max_sales.index.isin(top_shops_max.index)]

,date_block_num
item_id,
0,20
2,22
3,19
4,20
5,23
...,...
22157,5
22159,29
22160,8


# Задание 4

1. Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.







In [ ]:
#mean price for the least popular categories
shops = items.merge(sales, on='item_id')
shops[shops.category_id.isin(min.index)].groupby('date_block_num').mean().drop(columns=['item_id', 'item_cnt_day', 'shop_id'])

,category_id,item_price
date_block_num,,
3,10.0,1998.0
18,51.0,129.0


In [ ]:
#mean price for the most popular category
shops[shops.category_id.isin(max.index)].groupby('date_block_num').mean().drop(columns=['item_id', 'item_cnt_day', 'shop_id'])

,category_id,item_price
date_block_num,,
0,40.0,244.485942
1,40.0,252.174936
2,40.0,260.645016
3,40.0,254.852406
4,40.0,247.627583
5,40.0,245.860855
6,40.0,236.812052
7,40.0,234.470179
8,40.0,241.818554


In [ ]:
#median price for the least popular categories
shops[shops.category_id.isin(min.index)].groupby('date_block_num').median().drop(columns=['item_id', 'item_cnt_day', 'shop_id'])

,category_id,item_price
date_block_num,,
3,10.0,1998.0
18,51.0,129.0


In [ ]:
#median price for the most popular category
shops[shops.category_id.isin(max.index)].groupby('date_block_num').median().drop(columns=['item_id', 'item_cnt_day', 'shop_id'])

,category_id,item_price
date_block_num,,
0,40.0,199.00
1,40.0,203.00
2,40.0,231.00
3,40.0,199.00
4,40.0,199.00
5,40.0,199.00
6,40.0,149.00
7,40.0,149.00
8,40.0,198.00


2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
months = shops.groupby('date_block_num').mean().drop(columns=['item_id', 'item_cnt_day', 'shop_id', 'category_id'])


In [ ]:
months = shops.groupby('date_block_num').mean().drop(columns=['item_id', 'item_cnt_day', 'shop_id', 'category_id'])
needed = []
for i in list(months.index)[1:-1]:
  if (months['item_price'][i] < months['item_price'][i-1]) and (months['item_price'][i] < months['item_price'][i+1]):
    needed.append(i)

In [ ]:
thirteen = []
fourteen = []
fifteen = []
for i in needed:
  if i < 12:
    thirteen.append(i)
  elif i > 11 & i < 24:
    fourteen.append(i)
  else:
    fifteen.append(i)

print('2013:', thirteen, '\n2014:', fourteen, '\n2015:', fifteen)


2013: [4, 6] 
2014: [12, 15, 18, 25, 31] 
2015: []
